<a href="https://colab.research.google.com/github/panka-z/DataScience/blob/master/resume_model_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# #Mounting google drive into colab
#from google.colab import drive
#drive.mount('/content/gdrive')

print(len(open(filepath,'r',encoding="utf8").readlines()))

1513


In [24]:
#------------imports ---------------------------------
from __future__ import unicode_literals, print_function
import json
import pprint
import re
import plac
import random
from pathlib import Path
import spacy
import time
from spacy.util import minibatch, compounding

#--------------initial setup ------------------------------

start_time = time.time()

#debug variable
temp=[]

# file path from mounted gdrive
filepath="gdrive/My Drive/Demo Document Annotations.json"
output_dir="gdrive/My Drive/NLP_Model/"

#filepath="C:/Users/User/Downloads/Demo Document Annotations.json"
#output_dir="C:\Suven\AdvML\Resume_Summarisation_Project"
#output_dir="C:/Users/User/Downloads/"

# training data
TRAIN_DATA = [
	("Who is Shaka Khan? , I am Pankaj", {"entities": [(7, 17, "PERSON"),(26, 32, "PERSON")]}),
	("I like London and Berlin.", {"entities": [(7, 13, "LOC"), (18, 24, "LOC")]}),
]

TEST_DATA = [
	("I am Peter",{}),
	("I like Mumbai and Calcutta",{})
]

TRAIN_DATA[0][0][26:32]

# # uploading local files onto colab
# from google.colab import files
# uploaded = files.upload()

# #Mounting google drive into colab
# from google.colab import drive
# drive.mount('/content/gdrive')

# with open("gdrive/My Drive/Demo Document Annotations1.json",'rb') as f:
  # for line in f:
	# print(line)
	
training_data = []	
resume_count=len(list(open(filepath,'r',encoding="utf8").readlines()))
print('Total available resumes => ',resume_count)

#------------------------------------------------------


def load_data():
	print('loading training data ...')
	lines=open(filepath,'r',encoding="utf8").readlines()	
	for i,line in enumerate(lines):
		if i>440:
			break
		data = json.loads(line)
		text = data['content']
		entities = []
		if data['annotation']:
			for annotation in data['annotation']:
				#only a single point in text annotation.
				point = annotation['points'][0]
				labels = [ str(i).upper() for i in annotation['label']]
				#print(labels)
				# handle both list of labels or a single label.
				if not isinstance(labels, list):
					labels = [labels]

				for label in labels:
					#dataturks indices are both inclusive [start, end] but spacy is not [start, end)
					entities.append((point['start'], point['end'] + 1 ,label))

			training_data.append((text, {"entities" : entities}))
	print('loaded training data ...')


load_data()
TRAIN_DATA=training_data	
# test the saved model
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)
# Check the classes have loaded back consistently
#assert nlp2.get_pipe("ner").move_names == list(ner.move_names)
print('-'*100,'\n')
resumes_summary=[]

for text,_ in TRAIN_DATA:
	doc2 = nlp2(text)
	#pprint.pprint(doc2)
	#print('Predicted entities > ','\n')
	summary={}
	for ent in doc2.ents:
		if ent.label_:
			# used set to avoid duplicate entities
			summary.setdefault(ent.label_,set())
			summary[ent.label_].add(ent.text)
			#print(ent.label_, ent.text)
	resumes_summary.append(summary) 
 
print('Prediction of the loaded resumes generated...','\n')
for resume in resumes_summary:
	for key,value in resume.items():
		print(key,':','\n',','.join(value))
	print('-'*100)

print('Execution completed in => ',time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))

Total available resumes =>  1513
loading training data ...
loaded training data ...
Loading from gdrive/My Drive/NLP_Model/
---------------------------------------------------------------------------------------------------- 

Prediction of the loaded resumes generated... 

NAME : 
 Abhik Banerjee
DEGREE : 
 Master of Science in  Computer Science           	   
	     Cincinnati,OH,Bachelor of Engineering
COLLEGE NAME : 
 Orissa, India,Cincinnati Children’s Hospital Medical Center BMI R &D
LOCATION : 
 Bangalore
----------------------------------------------------------------------------------------------------
NAME : 
 Christof Meerwald
DESIGNATION : 
 Senior Software Developer,Senior Software Engineer Alcatel-Lucent (formerly Lucent Technologies) 12/2005 – 12/2007
– Worked in a global development team on telecom network management software for optical net-,Senior Software Engineer Emnico Technologies Ltd./Emnico Services Ltd. 1/2008 –
--------------------------------------------------

NameError: ignored

In [0]:
pprint.pprint(resumes_summary)